In [1]:
import os
import regex as re
import pandas as pd
import spacy
from pathlib import Path
from collections import Counter

In [2]:
# Read in the text file and make sure it's encoded in UTF-8. 
# Please change it to your respective file path.

text = open("corpus/Potter_Rabbit_anno_20.01.xml", mode='r', encoding='utf-8-sig').read()
#text[:100]

In [3]:
# Regular expression pattern to match <sound>...</sound>
pattern = r'(<sound>[A-Za-z]+</sound>)'

# Find all matches in the text
sound_words = re.findall(pattern, text)

# Print the list of matches
#print(sound_words)

In [4]:
pattern = r'(?s)\<\?xml version="1.0" encoding="UTF-8"\?>.*?<body>'
pattern2 = r'(?s)\</body>.*?</TEI>'

# Use re.sub to remove the matched portion from the text
text_without_match = re.sub(pattern, '', text)
text_without_match = re.sub(pattern2, '', text_without_match)

#print(text_without_match)

In [5]:
text_without_match = re.sub('\n+', ' ', text_without_match)
text_without_match = re.sub('\s+', ' ', text_without_match)
text_without_match

" TALE OF PETER RABBIT Beatrix Potter Once upon a time there were four little Rabbits and their names were -- Flopsy Mopsy Cotton-tail and Peter. They lived with their Mother in a sand-bank underneath the root of a very big fir-tree. Now my dears said old Mrs. Rabbit one morning you may go into the fields or down the lane but do n't go into Mr. McGregor s garden: your Father had an accident there he was put in a pie by Mrs. McGregor. Now run along and do n't get into mischief. I am going out. Then old Mrs. Rabbit took a basket and her umbrella and went through the wood to the baker s. She bought a loaf of brown bread and five currant buns. Flopsy Mopsy and Cottontail who were good little bunnies went down the lane to gather blackberries: But Peter who was very naughty ran straight away to Mr. McGregor s garden and squeezed under the gate! First he ate some lettuces and some French beans and then he ate some radishes And then feeling rather sick he went to look for some parsley. But rou

In [6]:
import spacy
from spacy.util import compile_infix_regex

# Load the SpaCy English model
nlp = spacy.load("en_core_web_sm")

# Sample text
input_text = text_without_match

# Define custom infix patterns to preserve "<sound>...</sound>" as one token
infixes = [
    r"(?<=\</sound>)(?=[\s\.,;:!?])",  # Split after '</sound>' followed by punctuation or space
    r"(?<=\s|[\<\>\.,;:!?])(?=\<sound\>)",  # Split before '<sound>' followed by punctuation or space
]

# Compile infix regex patterns
infix_re = compile_infix_regex(infixes)

# Set the custom infix regex as the tokenizer's infix pattern finder
nlp.tokenizer.infix_finditer = infix_re.finditer

# Tokenize the input text
doc = nlp(input_text)

# Get the tokens
tokens = [token.text for token in doc]

# Print the tokens
#print(tokens)

/Users/guhr/opt/anaconda3/lib/python3.9/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.6.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [7]:
df = pd.DataFrame(tokens, columns=['tokens'])
#df

In [8]:
#filtered_df['sound_annotation'] = filtered_df['tokens'].str.match(r'sound>(.*?)</sound')
df['sound_annotation'] = df['tokens'].str.extract(r'(sound>((.)*)</sound)')[1]

In [9]:
length = len(tokens)
#print(length)
list = ['O']* length 
#len(list)

In [10]:
# Show and controll the data frame before saving it to a csv-file.
df['annotation'] = list
#df.head(n = 10)

In [11]:
# Iterate over the "tokens" column and update the "annotation" column
for index, row in df.iterrows():
    token_text = row["tokens"]
    if re.search(r'sound>(.*?)</sound', token_text):
        df.at[index, "annotation"] = "sound"


In [12]:
# Iterate over the DataFrame and update the "tokens" column
for index, row in df.iterrows():
    token_text = row["tokens"]
    sound_annotation = row["sound_annotation"]
    match = re.search(r'sound>(.*?)</sound', token_text)
    if match:
        df.at[index, "tokens"] = sound_annotation

In [13]:
# Drop rows with empty cells in the "tokens" column
df = df[df["tokens"].str.strip() != ""]
df = df[~df["tokens"].str.contains("<|>")]

# Reset the index to have continuous row numbers
df.reset_index(drop=True, inplace=True)

In [14]:
del df['sound_annotation']

In [15]:
#df

In [16]:
# Save the DataFrame to a CSV file
df.to_csv("611Potter1902_anno.csv", index=False)